In [1]:
import numpy as  np 
from dataclasses import dataclass

In [2]:
@dataclass
class Direction():
    UP = 1
    DOWN = 2
    RIGHT = 3
    LEFT = 4

@dataclass
class Player1Direction():
    STATIC = 0
    UP = 1
    DOWN = 2

@dataclass
class Player2Direction():
    STATIC = 0
    UP = 1
    DOWN = 2

In [3]:
# Starting direction
direction = [Direction.DOWN, Direction.LEFT]
direction

[2, 4]

In [4]:
# Starting player direction
player1_direction = Player1Direction.STATIC
player2_direction = Player2Direction.STATIC


In [5]:
# CONFIG
BALL = 5

PLAYER_1 = 1
PLAYER_2 = 2
BOARD_SIZE = 10
MID = int(BOARD_SIZE/2)

WALL = 10
WALL_SIDES = 20

In [6]:
def create_map():
    map = np.zeros((BOARD_SIZE,BOARD_SIZE),dtype=np.int8)
    # Add walls
    map[[0,-1],2:-2] = WALL
    map[:,[0,-1]] = WALL_SIDES
    #map[0,1],map[-1,1],map[0,-2],map[-1,-2] = 100,100,100,100

    # Add ball
    map[MID+1,MID+1] = 5

    # Add players
    #map[-3:-1,1] = PLAYER_1
    map[MID-1:MID+1,1] = PLAYER_1
    map[MID-1:MID+1,-2] = PLAYER_2
    return map

map = create_map()
map

array([[20,  0, 10, 10, 10, 10, 10, 10,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  1,  0,  0,  0,  0,  0,  0,  2, 20],
       [20,  1,  0,  0,  0,  0,  0,  0,  2, 20],
       [20,  0,  0,  0,  0,  0,  5,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0, 10, 10, 10, 10, 10, 10,  0, 20]], dtype=int8)

In [7]:
# Remove hardcoded when tested
def game_logic(direction):
    # Game over check
    if 25 in map:
        print("Game over!")
        return False
    
    # Check wall hits
    if 15 in map:
        if Direction.DOWN == direction[0]: direction[0] = Direction.UP
        else: direction[0] = Direction.DOWN      
        return direction

    if 5 == map[0,1] or 5 == map[-1,1] or 5 == map[0,-2] or 5 == map[-1,-2]: # Edge case
        print("Edge case!")
        if Direction.DOWN == direction[0]: direction[0] = Direction.UP
        else: direction[0] = Direction.DOWN
        if Direction.RIGHT == direction[1]: direction[1] = Direction.LEFT
        else: direction[1] = Direction.RIGHT
        return direction
    elif 6 in map or 7 in map:
        if Direction.RIGHT == direction[1]: direction[1] = Direction.LEFT
        else: direction[1] = Direction.RIGHT
        return direction
    
    return direction
    


In [8]:
def get_ball_position(map):
    for i in [5,6,7,15]:#,101,102]:
        if i in map: 
            index_ball = np.where(map == i)
            return map, index_ball#([int(index_ball[0]),int(index_ball[1])])


In [9]:
def move_ball(map, direction, index_ball):
    if Direction.RIGHT == direction[1] and Direction.DOWN == direction[0]: map[index_ball[0]+1,index_ball[1]+1] += 5
    if Direction.LEFT == direction[1] and Direction.DOWN == direction[0]: map[index_ball[0]+1,index_ball[1]-1] += 5
    if Direction.RIGHT == direction[1] and Direction.UP == direction[0]: map[index_ball[0]-1,index_ball[1]+1] += 5
    if Direction.LEFT == direction[1] and Direction.UP == direction[0]: map[index_ball[0]-1,index_ball[1]-1] += 5

    # Recover old positions when hitting somthing
    if map[index_ball[0],index_ball[1]][0] == 5:   
        map[index_ball[0],index_ball[1]] = 0

    elif map[index_ball[0],index_ball[1]][0] == 6:   
        map[index_ball[0],index_ball[1]] = 1
    
    elif map[index_ball[0],index_ball[1]][0] == 7:   
        map[index_ball[0],index_ball[1]] = 2
        
    else:
        map[index_ball[0],index_ball[1]] = 10

    return map

In [10]:

direction = game_logic(direction)
if direction != False:
    map, index_ball = get_ball_position(map)
    map = move_ball(map, direction, index_ball)
    
direction, map

([2, 4],
 array([[20,  0, 10, 10, 10, 10, 10, 10,  0, 20],
        [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
        [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
        [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
        [20,  1,  0,  0,  0,  0,  0,  0,  2, 20],
        [20,  1,  0,  0,  0,  0,  0,  0,  2, 20],
        [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
        [20,  0,  0,  0,  0,  5,  0,  0,  0, 20],
        [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
        [20,  0, 10, 10, 10, 10, 10, 10,  0, 20]], dtype=int8))

In [11]:
def move_player(map, player, player_direction):
    " Move player x up or down"

    if player == PLAYER_1:  
        if len(np.where(map[0,1] == 1)[0]) > 0 and player_direction == 1:
            #print("Invalid move")
            return map
        elif len(np.where(map[-1,1] == 1)[0]) > 0 and player_direction == 2:   
            #print("Invalid move")
            return map

    else:
        if len(np.where(map[0,-2] == 2)[0]) > 0 and player_direction == 1:
            #print("Invalid move")
            return map
        elif len(np.where(map[-1,-2] == 2)[0]) > 0 and player_direction == 2:   
            #print("Invalid move")
            return map

    # Move player if not static
    if player_direction > 0 :
        player_position = np.where(map == player)
        map[player_position[0],player_position[1][0]] = 0 # Clear old position
        if player_direction == 1: map[player_position[0]-1,player_position[1][0]] = player
        if player_direction == 2: map[player_position[0]+1,player_position[1][0]] = player

    return map

player1_direction = Player1Direction.UP
player2_direction = Player1Direction.DOWN
map = move_player(map, PLAYER_1, player1_direction)
map = move_player(map, PLAYER_2, player2_direction)
map

array([[20,  0, 10, 10, 10, 10, 10, 10,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  1,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  1,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  2, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  2, 20],
       [20,  0,  0,  0,  0,  5,  0,  0,  0, 20],
       [20,  0,  0,  0,  0,  0,  0,  0,  0, 20],
       [20,  0, 10, 10, 10, 10, 10, 10,  0, 20]], dtype=int8)